#**Kubernetes homework**

In [8]:
!pip install -q onnx onnxruntime

In [2]:
import numpy as np
import onnx
import onnxruntime as ort

from io import BytesIO
from urllib import request
from PIL import Image
from torchvision import transforms

In [9]:
!wget -q https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
!wget -q https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

###**<font color='red'>Question 1</font>**
To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:
- <font color='green'>output</font> ✅
- sigmoid
- softmax
- prediction

In [4]:
model = onnx.load("hair_classifier_v1.onnx")
graph = model.graph

print("Inputs:")
for inp in graph.input:
    print("-", inp.name)

print("\nOutputs:")
for out in graph.output:
  print("-", out.name)

Inputs:
- input

Outputs:
- output


###**<font color='red'>Question 2</font>**
Based on the previous homework, what should be the target size for the image?

- 64x64
- 128x128
- <font color='green'>200x200</font> ✅
- 256x256

###**<font color='red'>Question 3</font>**
Now we need to turn the image into numpy array and pre-process it.

After the pre-processing, what's the value in the first pixel, the R channel?
- -10.73
- <font color='green'>-1.073</font> ✅
- 1.073
- 10.73

In [5]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [6]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = download_image(url)
img = prepare_image(img, (200, 200))
tensor = train_transforms(img)
array = tensor.numpy()
print("First R pixel:", array[0, 0, 0])

First R pixel: -1.073294


###**<font color='red'>Question 4</font>**
Now let's apply this model to this image. What's the output of the model?

- <font color='green'>0.09</font> ✅
- 0.49
- 0.69
- 0.89

In [7]:
x = tensor.numpy()[None, :, :, :]

session = ort.InferenceSession("hair_classifier_v1.onnx")
input_name = session.get_inputs()[0].name

output = session.run(None, {input_name: x})[0]
prediction = float(output[0][0])

print("Model output:", round(prediction, 2))

Model output: 0.09


###**<font color='red'>Question 5</font>**
Download the base image agrigorev/model-2025-hairstyle:v1.

So what's the size of this base image?

- 88 Mb
- 208 Mb
- <font color='green'>608 Mb</font> ✅
- 1208 Mb

In [ ]:
!docker pull agrigorev/model-2025-hairstyle:v1
!docker images

###**<font color='red'>Question 6</font>**
Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is hair_classifier_empty.onnx and it's in the current workdir in the image (see the Dockerfile above for the reference). The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

- -1.0
- -0.10
- 0.10
- 1.0

In [ ]:
# docker build -t hairstyle-lambda .
# docker run -p 8080:8080 hairstyle-lambda

In [ ]:
!curl -X POST \
  "http://localhost:8080/2015-03-31/functions/function/invocations" \
  -d '{"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'